**Movie Recommendation System**

> **Recommender System** is a system that seeks to predict or filter preferences according to the user’s choices. Recommender systems are utilized in a variety of areas including movies, music, news, books, research articles, search queries, social tags, and products in general. Recommender systems produce a list of recommendations in any of the two ways –

> **Collaborative filtering**: Collaborative filtering approaches build a model from the user’s past behavior (i.e., items purchased or searched by the user) as well as similar decisions made by other users. This model is then used to predict items (or ratings for items) that users may have an interest in.

> **Content-based filtering**: Content-based filtering approaches use a series of discrete characteristics of an item in order to recommend additional items with similar properties. Content-based filtering methods are totally based on a description of the item and a profile of the user’s preferences. It recommends items based on the user’s past preferences.

> Let’s develop a basic recommendation system by suggesting items that are most similar to a particular item, in this case, movies. It just tells what movies/items are most similar to the user’s movie choice.

**Import Library**

In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**Import Dataset**

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/YBI-Foundation/Dataset/refs/heads/main/Movies%20Recommendation.csv')

In [3]:
df.head()

,Movie_ID,Movie_Title,Movie_Genre,Movie_Language,Movie_Budget,Movie_Popularity,Movie_Release_Date,Movie_Revenue,Movie_Runtime,Movie_Vote,...,Movie_Homepage,Movie_Keywords,Movie_Overview,Movie_Production_House,Movie_Production_Country,Movie_Spoken_Language,Movie_Tagline,Movie_Cast,Movie_Crew,Movie_Director
0,1,Four Rooms,Crime Comedy,en,4000000,22.876230,09-12-1995,4300000,98.0,6.5,...,NaN,hotel new year's eve witch bet hotel room,It's Ted the Bellhop's first night on the job....,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,"[{'name': 'Allison Anders', 'gender': 1, 'depa...",Allison Anders
1,2,Star Wars,Adventure Action Science Fiction,en,11000000,126.393695,25-05-1977,775398007,121.0,8.1,...,http://www.starwars.com/films/star-wars-episod...,android galaxy hermit death star lightsaber,Princess Leia is captured and held hostage by ...,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,"[{'name': 'George Lucas', 'gender': 2, 'depart...",George Lucas
2,3,Finding Nemo,Animation Family,en,94000000,85.688789,30-05-2003,940335536,100.0,7.6,...,http://movies.disney.com/finding-nemo,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...","[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
3,4,Forrest Gump,Comedy Drama Romance,en,55000000,138.133331,06-07-1994,677945399,142.0,8.2,...,NaN,vietnam veteran hippie mentally disabled runni...,A man with a low IQ has accomplished great thi...,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,"[{'name': 'Alan Silvestri', 'gender': 2, 'depa...",Robert Zemeckis
4,5,American Beauty,Drama,en,15000000,80.878605,15-09-1999,356296601,122.0,7.9,...,http://www.dreamworks.com/ab/,male nudity female nudity adultery midlife cri...,"Lester Burnham, a depressed suburban father in...","[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4760 entries, 0 to 4759
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Movie_ID                  4760 non-null   int64  
 1   Movie_Title               4760 non-null   object 
 2   Movie_Genre               4760 non-null   object 
 3   Movie_Language            4760 non-null   object 
 4   Movie_Budget              4760 non-null   int64  
 5   Movie_Popularity          4760 non-null   float64
 6   Movie_Release_Date        4760 non-null   object 
 7   Movie_Revenue             4760 non-null   int64  
 8   Movie_Runtime             4758 non-null   float64
 9   Movie_Vote                4760 non-null   float64
 10  Movie_Vote_Count          4760 non-null   int64  
 11  Movie_Homepage            1699 non-null   object 
 12  Movie_Keywords            4373 non-null   object 
 13  Movie_Overview            4757 non-null   object 
 14  Movie_Pr

In [5]:
df.shape

(4760, 21)

In [6]:
df.columns

Index(['Movie_ID', 'Movie_Title', 'Movie_Genre', 'Movie_Language',
       'Movie_Budget', 'Movie_Popularity', 'Movie_Release_Date',
       'Movie_Revenue', 'Movie_Runtime', 'Movie_Vote', 'Movie_Vote_Count',
       'Movie_Homepage', 'Movie_Keywords', 'Movie_Overview',
       'Movie_Production_House', 'Movie_Production_Country',
       'Movie_Spoken_Language', 'Movie_Tagline', 'Movie_Cast', 'Movie_Crew',
       'Movie_Director'],
      dtype='object')

**Get Feature Selection**

In [7]:
df_features = df[['Movie_Genre','Movie_Keywords','Movie_Tagline','Movie_Cast','Movie_Director']].fillna('')

In [8]:
df_features.shape

(4760, 5)

In [9]:
df_features

,Movie_Genre,Movie_Keywords,Movie_Tagline,Movie_Cast,Movie_Director
0,Crime Comedy,hotel new year's eve witch bet hotel room,Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,Allison Anders
1,Adventure Action Science Fiction,android galaxy hermit death star lightsaber,"A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,George Lucas
2,Animation Family,father son relationship harbor underwater fish...,"There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,Andrew Stanton
3,Comedy Drama Romance,vietnam veteran hippie mentally disabled runni...,"The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,Robert Zemeckis
4,Drama,male nudity female nudity adultery midlife cri...,Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,Sam Mendes
...,...,...,...,...,...
4755,Horror,,The hot spot where Satan's waitin'.,Lisa Hart Carroll Michael Des Barres Paul Drak...,Pece Dingo
4756,Comedy Family Drama,,It’s better to stand out than to fit in.,Roni Akurati Brighton Sharbino Jason Lee Anjul...,Frank Lotito
4757,Thriller Drama,christian film sex trafficking,She never knew it could happen to her...,Nicole Smolen Kim Baldwin Ariana Stephens Brys...,Jaco Booyens
4758,Family,,,,


In [10]:
x = df_features['Movie_Genre'] + ' ' + df_features['Movie_Keywords'] + ' ' + df_features['Movie_Tagline'] + ' ' + df_features['Movie_Cast'] + ' ' + df_features['Movie_Director']

In [11]:
x

,0
0,Crime Comedy hotel new year's eve witch bet ho...
1,Adventure Action Science Fiction android galax...
2,Animation Family father son relationship harbo...
3,Comedy Drama Romance vietnam veteran hippie me...
4,Drama male nudity female nudity adultery midli...
...,...
4755,Horror The hot spot where Satan's waitin'. Li...
4756,Comedy Family Drama It’s better to stand out ...
4757,Thriller Drama christian film sex trafficking ...
4758,Family


In [12]:
x.shape

(4760,)

**Get Feature Text Conversion to Tokens**

In [13]:
tfidf = TfidfVectorizer()

In [14]:
x = tfidf.fit_transform(x)

In [15]:
x.shape

(4760, 17258)

In [16]:
print(x)

  (0, 3583)	0.06486754376295062
  (0, 3240)	0.04527089872278055
  (0, 7213)	0.25146675849405775
  (0, 10898)	0.17625708810661284
  (0, 17052)	0.26079573581490934
  (0, 5059)	0.29553419178998613
  (0, 16862)	0.12768803549311025
  (0, 1595)	0.15687561633854538
  (0, 13052)	0.1465525095337543
  (0, 15708)	0.17654247479915475
  (0, 11362)	0.18801785343006192
  (0, 6463)	0.18801785343006192
  (0, 5662)	0.1465525095337543
  (0, 13467)	0.19712637387361423
  (0, 12731)	0.19712637387361423
  (0, 614)	0.07642616241686973
  (0, 11244)	0.08262965296941757
  (0, 9206)	0.15186283580984414
  (0, 1495)	0.19712637387361423
  (0, 7454)	0.14745635785412262
  (0, 7071)	0.19822417598406614
  (0, 5499)	0.11454057510303811
  (0, 3878)	0.11998399582562203
  (0, 11242)	0.07277788238484746
  (0, 15219)	0.09800472886453934
  :	:
  (4757, 3485)	0.199161573117024
  (4757, 1184)	0.18890726729447022
  (4757, 14568)	0.24255077606762876
  (4757, 15508)	0.24255077606762876
  (4757, 5802)	0.24255077606762876
  (4757, 81

In [17]:
similarity_score = cosine_similarity(x)

In [18]:
similarity_score

array([[1.        , 0.01351235, 0.03570468, ..., 0.        , 0.        ,
        0.        ],
       [0.01351235, 1.        , 0.00806674, ..., 0.        , 0.        ,
        0.        ],
       [0.03570468, 0.00806674, 1.        , ..., 0.        , 0.08014876,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.08014876, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [19]:
similarity_score.shape

(4760, 4760)

**Get Movie name as a Input fromm User and validate for closet Spelling**

In [20]:
Favourite_Movie_Name = input('Enter your favourite movie name: ')

Enter your favourite movie name: Family


In [21]:
All_Movie_Title_List = df['Movie_Title'].tolist()

In [22]:
Movie_Recommendation = difflib.get_close_matches(Favourite_Movie_Name, All_Movie_Title_List)
print(Movie_Recommendation)

['Frailty', 'Family Plot', 'The Family Man']


In [23]:
close_match = Movie_Recommendation[0]
print(close_match)

Frailty


In [24]:
index_of_close_match_movie = df[df.Movie_Title == close_match]['Movie_ID'].values[0]
print(index_of_close_match_movie)

2097


In [27]:
Recommendation_Score = list(enumerate(similarity_score[index_of_close_match_movie]))

In [28]:
print(Recommendation_Score)

[(0, 0.029892727213866675), (1, 0.0), (2, 0.031302204708425416), (3, 0.0027940145067739475), (4, 0.003353409758805672), (5, 0.012080240120488465), (6, 0.021965056287314814), (7, 0.002283125213724728), (8, 0.003451615958290116), (9, 0.0), (10, 0.009448468547008693), (11, 0.003291190974087277), (12, 0.025194804654685748), (13, 0.0), (14, 0.0), (15, 0.02176163083591936), (16, 0.0026490206812606666), (17, 0.02168613448669678), (18, 0.03193124318874002), (19, 0.03673888722351705), (20, 0.035980210288131635), (21, 0.012880112209127357), (22, 0.0), (23, 0.024586194619973804), (24, 0.0554102103604163), (25, 0.03640361612348495), (26, 0.003109737332384813), (27, 0.016999466627475694), (28, 0.0), (29, 0.0030265930319477705), (30, 0.013749006430693567), (31, 0.002989039742391375), (32, 0.002787123144268193), (33, 0.0028087506013468296), (34, 0.014551343437118171), (35, 0.024710407742360926), (36, 0.0), (37, 0.02814409962630985), (38, 0.045364925698499314), (39, 0.010850142608833017), (40, 0.05927

In [29]:
len(Recommendation_Score)

4760

**Get ALL Movies sort based on Recommendation Score wrt Favourite Movie**

In [31]:
sorted_Similar_Score = sorted(Recommendation_Score, key = lambda x:x[1], reverse = True)
print(sorted_Similar_Score)

[(2097, 1.0), (1865, 0.20270287921408287), (2635, 0.17535182728271942), (1958, 0.16646142649882523), (1796, 0.15522383361160902), (145, 0.14934000368453973), (1191, 0.14797610492953372), (112, 0.12829561060079597), (48, 0.12562751557836116), (3616, 0.12150704853658896), (1275, 0.12046897077771979), (1344, 0.12019319059100211), (523, 0.11969350039134714), (1716, 0.11852236260819327), (676, 0.11681739309779389), (1121, 0.11522741033561375), (801, 0.11479747119699674), (1885, 0.11194373310380708), (3766, 0.11162098978725463), (3730, 0.11146993404445854), (1334, 0.11009192688725009), (360, 0.1091972920142074), (2606, 0.10897825069507312), (1325, 0.10888192879808192), (1698, 0.10862174717182904), (1141, 0.10758719460961029), (228, 0.10705121690923244), (3466, 0.1054303336423417), (4446, 0.104376594112112), (2123, 0.10426754603229313), (1356, 0.10306435596840954), (2572, 0.10283430550317768), (580, 0.10280108030463891), (4225, 0.10256964838245658), (4531, 0.10239887611650599), (2486, 0.10233

In [32]:
print('top 30 movies suggested for you: \n')

i = 1

for movie in sorted_Similar_Score:
  index = movie[0]
  title_from_index = df[df.index == index]['Movie_Title'].values[0]
  if (i<31):
    print(i, '.', title_from_index)
    i+=1

top 30 movies suggested for you: 

1 . Sea of Love
2 . City Hall
3 . The Jerky Boys
4 . Domestic Disturbance
5 . Basic
6 . Insomnia
7 . The Insider
8 . The Godfather: Part III
9 . Scarface
10 . Another Happy Day
11 . Donnie Brasco
12 . Analyze This
13 . Secret Window
14 . Frequency
15 . L.A. Confidential
16 . Bringing Out the Dead
17 . 88 Minutes
18 . The Merchant of Venice
19 . The Son of No One
20 . Jack and Jill
21 . Glengarry Glen Ross
22 . Se7en
23 . Edmond
24 . The Bone Collector
25 . Drop Dead Gorgeous
26 . Mercury Rising
27 . Basquiat
28 . Man on a Ledge
29 . Danny Collins
30 . Factory Girl


**Top 10 Movie recommendation system**

In [33]:
Movie_name = input('Enter your favourite movie name: ')

list_of_all_titles = df['Movie_Title'].tolist()

Find_Close_Match = difflib.get_close_matches(Movie_name, list_of_all_titles)

Close_Match = Find_Close_Match[0]

Index_of_Movie = df[df.Movie_Title == Close_Match]['Movie_ID'].values[0]

Recommendation_Score = list(enumerate(similarity_score[Index_of_Movie]))

sorted_Similar_Movies = sorted(Recommendation_Score, key = lambda x:x[1], reverse = True)

print('Top 10 movies suggested for you: \n')

i = 1

for movie in sorted_Similar_Movies:
  index = movie[0]
  title_from_index = df[df.Movie_ID == index]['Movie_Title'].values
  if (i<11):
    print(i, '.', title_from_index)
    i+=1


Enter your favourite movie name: avtaar
Top 10 movies suggested for you: 

1 . ['Avatar']
2 . ['The Girl on the Train']
3 . ['Act of Valor']
4 . ['Donnie Darko']
5 . ['Precious']
6 . ['Freaky Friday']
7 . ['The Opposite Sex']
8 . ['Heaven is for Real']
9 . ['Run Lola Run']
10 . ['Elizabethtown']


**Explaination**
> The project is a Content-Based Movie Recommendation System that uses metadata like genres, keywords, cast, and directors to suggest movies similar to a user’s favorite. It preprocesses the data by combining relevant features, applies the TF-IDF technique for vectorizing text data, and calculates movie similarities using cosine similarity. Users can input a movie title, and the system finds the closest match, then recommends the most similar movies. The project highlights data preprocessing, natural language processing, and recommendation system techniques. It is an interactive and scalable solution for personalized movie recommendations.

